In [1]:
!pip install optuna

In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-10-25 12:19:24,470] A new study created in memory with name: no-name-8ff0f0d6-720a-47de-afa4-75a26a1093a7
[I 2025-10-25 12:19:25,395] Trial 0 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 145, 'max_depth': 11}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-10-25 12:19:26,097] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 115, 'max_depth': 13}. Best is trial 1 with value: 0.7728119180633147.
[I 2025-10-25 12:19:26,900] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 132, 'max_depth': 17}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-10-25 12:19:27,666] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 129, 'max_depth': 10}. Best is trial 2 with value: 0.7746741154562384.
[I 2025-10-25 12:19:28,068] Trial 4 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 65, 'max_depth': 16}. Best is trial 4 with value: 0.77653

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 117, 'max_depth': 15}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-10-25 12:20:01,323] A new study created in memory with name: no-name-373c8345-c204-4af6-888e-347f4dc7eb95
[I 2025-10-25 12:20:02,364] Trial 0 finished with value: 0.7783985102420857 and parameters: {'n_estimators': 165, 'max_depth': 16}. Best is trial 0 with value: 0.7783985102420857.
[I 2025-10-25 12:20:02,979] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 96, 'max_depth': 15}. Best is trial 0 with value: 0.7783985102420857.
[I 2025-10-25 12:20:03,537] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 98, 'max_depth': 5}. Best is trial 0 with value: 0.7783985102420857.
[I 2025-10-25 12:20:04,721] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 186, 'max_depth': 14}. Best is trial 0 with value: 0.7783985102420857.
[I 2025-10-25 12:20:05,201] Trial 4 finished with value: 0.756052141527002 and parameters: {'n_estimators': 91, 'max_depth': 3}. Best is trial 0 with value: 0.7783985102

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 128, 'max_depth': 16}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-10-25 12:20:38,721] A new study created in memory with name: no-name-3c6b635f-4bac-4f20-9875-d1f80e2563c6
[I 2025-10-25 12:20:39,232] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-25 12:20:40,110] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-25 12:20:40,413] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-10-25 12:20:40,987] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-10-25 12:20:41,544] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [17]:
!pip install cmaes

  Using cached cmaes-0.12.0-py3-none-any.whl.metadata (29 kB)
Using cached cmaes-0.12.0-py3-none-any.whl (64 kB)


In [18]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.CmaEsSampler())
study.optimize(objective, n_trials=50)

[I 2025-10-25 12:20:52,829] A new study created in memory with name: no-name-b0c8d2f0-b617-4411-a81c-10112ed3dba5
[I 2025-10-25 12:20:53,375] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 79, 'max_depth': 19}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-25 12:20:54,004] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 81, 'max_depth': 7}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-25 12:20:54,547] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 67, 'max_depth': 11}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-25 12:20:55,784] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 169, 'max_depth': 11}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-25 12:20:56,594] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 111, 'max_depth': 14}. Best is trial 4 with value: 0.77281191

In [19]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 131, 'max_depth': 12}


In [20]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.75


In [21]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_n_estimators,system_attrs_cma:generation,system_attrs_cma:optimizer:0,system_attrs_cma:optimizer:1,state
0,0,0.769088,2025-10-25 12:20:52.832279,2025-10-25 12:20:53.375767,0 days 00:00:00.543488,19,79,NaN,NaN,NaN,COMPLETE
1,1,0.767225,2025-10-25 12:20:53.376846,2025-10-25 12:20:54.003891,0 days 00:00:00.627045,7,81,0.0,NaN,NaN,COMPLETE
2,2,0.765363,2025-10-25 12:20:54.005320,2025-10-25 12:20:54.547862,0 days 00:00:00.542542,11,67,0.0,NaN,NaN,COMPLETE
3,3,0.767225,2025-10-25 12:20:54.548879,2025-10-25 12:20:55.784685,0 days 00:00:01.235806,11,169,0.0,NaN,NaN,COMPLETE
4,4,0.772812,2025-10-25 12:20:55.785926,2025-10-25 12:20:56.594099,0 days 00:00:00.808173,14,111,0.0,NaN,NaN,COMPLETE
5,5,0.761639,2025-10-25 12:20:56.595376,2025-10-25 12:20:57.419647,0 days 00:00:00.824271,9,123,0.0,NaN,NaN,COMPLETE
6,6,0.763501,2025-10-25 12:20:57.421189,2025-10-25 12:20:58.474069,0 days 00:00:01.052880,11,122,0.0,NaN,NaN,COMPLETE
7,7,0.769088,2025-10-25 12:20:58.476243,2025-10-25 12:20:59.523265,0 days 00:00:01.047022,8,134,1.0,8004954e060000000000008c0a636d6165732e5f636d61...,3200000000000000000000000000000000000000000000...,COMPLETE
8,8,0.770950,2025-10-25 12:20:59.525250,2025-10-25 12:21:00.800863,0 days 00:00:01.275613,15,149,1.0,NaN,NaN,COMPLETE
9,9,0.772812,2025-10-25 12:21:00.802110,2025-10-25 12:21:01.596956,0 days 00:00:00.794846,14,118,1.0,NaN,NaN,COMPLETE


In [22]:
study.sampler

## Optuna Visualizations

In [23]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [24]:
# 1. Optimization History
plot_optimization_history(study).show()

In [25]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [26]:
# 3. Slice Plot
plot_slice(study).show()

In [27]:
# 4. Contour Plot
plot_contour(study).show()

In [28]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [29]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [30]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [31]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-10-25 12:21:36,336] A new study created in memory with name: no-name-813095ea-52fb-44b2-b653-11b8d6133b0c
[I 2025-10-25 12:21:36,764] Trial 0 finished with value: 0.7672253258845437 and parameters: {'classifier': 'RandomForest', 'n_estimators': 68, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-10-25 12:21:37,163] Trial 1 finished with value: 0.7616387337057727 and parameters: {'classifier': 'RandomForest', 'n_estimators': 62, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-10-25 12:21:37,209] Trial 2 finished with value: 0.7076350093109869 and parameters: {'classifier': 'SVM', 'C': 5.737097686377093, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-10-25 12:21:37,959] Trial 3 finished with value: 0.7430167597765364 and parameters: {'classifier': 'R

In [32]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11793145480474733, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [33]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.767225,2025-10-25 12:21:36.339875,2025-10-25 12:21:36.764391,0 days 00:00:00.424516,NaN,False,RandomForest,NaN,NaN,NaN,10.0,3.0,6.0,68.0,COMPLETE
1,1,0.761639,2025-10-25 12:21:36.765439,2025-10-25 12:21:37.163730,0 days 00:00:00.398291,NaN,True,RandomForest,NaN,NaN,NaN,20.0,4.0,6.0,62.0,COMPLETE
2,2,0.707635,2025-10-25 12:21:37.164753,2025-10-25 12:21:37.209667,0 days 00:00:00.044914,5.737098,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.743017,2025-10-25 12:21:37.210785,2025-10-25 12:21:37.958949,0 days 00:00:00.748164,NaN,False,RandomForest,NaN,NaN,NaN,3.0,6.0,9.0,174.0,COMPLETE
4,4,0.716946,2025-10-25 12:21:37.960183,2025-10-25 12:21:38.017435,0 days 00:00:00.057252,78.490236,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2025-10-25 12:22:13.838945,2025-10-25 12:22:13.878078,0 days 00:00:00.039133,0.186423,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-10-25 12:22:13.879161,2025-10-25 12:22:13.916716,0 days 00:00:00.037555,0.303062,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.746741,2025-10-25 12:22:13.918019,2025-10-25 12:22:13.963451,0 days 00:00:00.045432,0.121346,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.789572,2025-10-25 12:22:13.964463,2025-10-25 12:22:13.999853,0 days 00:00:00.035390,0.147924,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [34]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 79
RandomForest        11
GradientBoosting    10
Name: count, dtype: int64

In [35]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.751024
RandomForest        0.764178
SVM                 0.774627
Name: value, dtype: float64

In [36]:
# 1. Optimization History
plot_optimization_history(study).show()

In [37]:
# 3. Slice Plot
plot_slice(study).show()

In [38]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [ ]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.1 MB/s eta 0:00:00


In [41]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-10-25 12:38:19,432] A new study created in memory with name: no-name-49922c8e-6ef7-41ca-a308-58b5e5597a6d


[0]	train-mlogloss:1.05188	eval-mlogloss:1.05784
[1]	train-mlogloss:0.94867	eval-mlogloss:0.94981
[2]	train-mlogloss:0.85926	eval-mlogloss:0.85215
[3]	train-mlogloss:0.79976	eval-mlogloss:0.78796
[4]	train-mlogloss:0.73940	eval-mlogloss:0.72729
[5]	train-mlogloss:0.67398	eval-mlogloss:0.65801
[6]	train-mlogloss:0.63164	eval-mlogloss:0.61751
[7]	train-mlogloss:0.60157	eval-mlogloss:0.58668
[8]	train-mlogloss:0.55370	eval-mlogloss:0.53422
[9]	train-mlogloss:0.52107	eval-mlogloss:0.50201
[10]	train-mlogloss:0.48300	eval-mlogloss:0.46113
[11]	train-mlogloss:0.45095	eval-mlogloss:0.42549
[12]	train-mlogloss:0.43919	eval-mlogloss:0.41421
[13]	train-mlogloss:0.40759	eval-mlogloss:0.37897
[14]	train-mlogloss:0.39843	eval-mlogloss:0.37272
[15]	train-mlogloss:0.38137	eval-mlogloss:0.35506
[16]	train-mlogloss:0.36495	eval-mlogloss:0.33950
[17]	train-mlogloss:0.34658	eval-mlogloss:0.32196
[18]	train-mlogloss:0.33387	eval-mlogloss:0.30901
[19]	train-mlogloss:0.32164	eval-mlogloss:0.29701
[20]	train

[I 2025-10-25 12:38:22,347] Trial 0 finished with value: 1.0 and parameters: {'lambda': 1.184525138592032e-08, 'alpha': 0.000125766016079723, 'eta': 0.08720432492028109, 'gamma': 1.3791502024418182e-08, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6163936156149469, 'colsample_bytree': 0.4581034127020779}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.88861	eval-mlogloss:0.86624
[1]	train-mlogloss:0.64796	eval-mlogloss:0.61602
[2]	train-mlogloss:0.49140	eval-mlogloss:0.45177
[3]	train-mlogloss:0.38611	eval-mlogloss:0.34179
[4]	train-mlogloss:0.33252	eval-mlogloss:0.28315
[5]	train-mlogloss:0.28968	eval-mlogloss:0.23782
[6]	train-mlogloss:0.25786	eval-mlogloss:0.20227
[7]	train-mlogloss:0.25752	eval-mlogloss:0.20171
[8]	train-mlogloss:0.24854	eval-mlogloss:0.19021
[9]	train-mlogloss:0.24643	eval-mlogloss:0.18453
[10]	train-mlogloss:0.24623	eval-mlogloss:0.18535
[11]	train-mlogloss:0.24536	eval-mlogloss:0.18265
[12]	train-mlogloss:0.24434	eval-mlogloss:0.18194
[13]	train-mlogloss:0.24407	eval-mlogloss:0.18139
[14]	train-mlogloss:0.24370	eval-mlogloss:0.18111
[15]	train-mlogloss:0.24370	eval-mlogloss:0.18130
[16]	train-mlogloss:0.24372	eval-mlogloss:0.18118
[17]	train-mlogloss:0.24368	eval-mlogloss:0.18128
[18]	train-mlogloss:0.24383	eval-mlogloss:0.18097
[19]	train-mlogloss:0.24320	eval-mlogloss:0.17905
[20]	train

[I 2025-10-25 12:38:23,785] Trial 1 finished with value: 1.0 and parameters: {'lambda': 2.132446602933135e-08, 'alpha': 1.6008550631491214e-07, 'eta': 0.2863302227269569, 'gamma': 0.0038611535667931987, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.6864422991563024, 'colsample_bytree': 0.5381650298115359}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.94946	eval-mlogloss:0.92829
[1]	train-mlogloss:0.75729	eval-mlogloss:0.72289
[2]	train-mlogloss:0.61607	eval-mlogloss:0.57582


[I 2025-10-25 12:38:41,846] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.96942	eval-mlogloss:0.96184
[1]	train-mlogloss:0.84176	eval-mlogloss:0.81855
[2]	train-mlogloss:0.71302	eval-mlogloss:0.68789


[I 2025-10-25 12:38:41,911] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 1.184525138592032e-08, 'alpha': 0.000125766016079723, 'eta': 0.08720432492028109, 'gamma': 1.3791502024418182e-08, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6163936156149469, 'colsample_bytree': 0.4581034127020779}
Best accuracy: 1.0


In [42]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()